# Gujarat Industry Area Location

In [96]:
import pandas as pd
import geocoder
import folium
from folium.features import DivIcon
from branca.element import Figure

# Load Data

## External Industry Area Data

In [ ]:
df_ind_area = pd.read_csv('../../data/gujarat_industry_data.csv')
df_ind_area

In [20]:
df_ind_area.Estate.unique()

array(['Sanand (Mini)', 'Vani', "Vatwa- Zone 'D'", 'Dhandhuka', 'Dholka',
       'Viramgam', 'Kerala', 'Kathawada I & II', 'Odhav', 'Naroda',
       'Vatwa', 'Sankheda', 'Pavi-jetpur', 'GTC Ltd.', 'Dabhoi', 'Ranoli',
       'Limbda (Appolo Tyre)', 'Por-Ramangamdi', 'Nandesari', 'Makarpura',
       'Vaghodia', 'P.C.C.', 'Savli', 'Thasra', 'Kapadwanj', 'Matar',
       'Nadiad-I', 'Nadiad- II', 'Balasinor', 'Sojitra', 'Umreth',
       'Petlad', 'Borsad', 'Khambat', 'V.U. Nagar', 'Bhat',
       'Gandhinagar-I', 'Gandhinagar-II', 'Dehgam', 'Mansa', 'Ranasan',
       'Gozaria', 'Pardi', 'Dharampur-II', 'Bhilad', 'Valsad', 'Umargam',
       'Sarigam', 'Vapi', 'Unai', 'Bilimora', 'Navsari', 'Dhrol',
       'Bhatia', 'Bhanvad', 'Jamkhambhalia', 'Arambhada', 'Jamnagar-I',
       'Jamnagar-II', 'Kotda-Sangani', 'Paddhari', 'Malia', 'Jasdan',
       'Bhaktinagar', 'Jetpur', 'Wankaner', 'Morbi', 'Dhoraji',
       'Gondal-I', 'Gondal-II', 'Rafaleshwar', 'Kuvadawa', 'Aji',
       'Lodhika', 'Valod', 

## Load Distance Matrix and Biomass History

In [ ]:
d_matrix = pd.read_csv('../../data/Distance_Matrix.csv')
df_fc = pd.read_csv('../../data/Biomass_History.csv')

print(d_matrix.iloc[:5, :5])
df_fc.head()

# Get Coordinates From Estate and District

In [84]:
# geocoder library is used to get the coordinates of the estates. 
# The method "osm" is used to get the coordinates from the OpenStreetMap API.

df_ind_area['Coords'] = None
df_ind_area.loc[:, ['Coords']] = df_ind_area[['Estate', 'District']].\
                                    apply(lambda x: geocoder.osm(f'{x[0]}, {x[1]}, Gujarat, India').latlng, axis=1)

In [85]:
df_ind_area.head()

,Sr.No.,Estate,District,Size in Ha.,Power Supply (MVA),Water Supply (MLD),Drainage (MLD),Latitude,Longitude,Coords
0,1,Sanand (Mini),Ahmedabad,1.8,0.50,2.70,0.00,None,None,None
1,2,Vani,Ahmedabad,2.0,2.00,0.20,0.00,None,None,"[23.0683251, 71.9814201]"
2,3,Vatwa- Zone 'D',Ahmedabad,6.3,2.00,0.43,0.00,None,None,None
3,4,Dhandhuka,Ahmedabad,23.0,1.00,0.01,0.00,None,None,"[22.3816365, 71.9791414]"
4,5,Dholka,Ahmedabad,42.0,2.20,0.20,0.00,None,None,"[22.726607, 72.4411863]"


Some of the places weren't found. For now, we'll drop them. They can be easily filled with some external info.

In [86]:
df_ind_area_nn = df_ind_area[~df_ind_area['Coords'].isna()]
df_ind_area_nn['Latitude'] = df_ind_area_nn['Coords'].apply(lambda x: x[0])
df_ind_area_nn['Longitude'] = df_ind_area_nn['Coords'].apply(lambda x: x[1])
df_ind_area_nn = df_ind_area_nn.drop(columns=['Coords'])
print(df_ind_area_nn.shape)
df_ind_area_nn.head()

(123, 9)


C:\Users\alber\AppData\Local\Temp\ipykernel_26972\1935755051.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ind_area_nn['Latitude'] = df_ind_area_nn['Coords'].apply(lambda x: x[0])
C:\Users\alber\AppData\Local\Temp\ipykernel_26972\1935755051.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ind_area_nn['Longitude'] = df_ind_area_nn['Coords'].apply(lambda x: x[1])


,Sr.No.,Estate,District,Size in Ha.,Power Supply (MVA),Water Supply (MLD),Drainage (MLD),Latitude,Longitude
1,2,Vani,Ahmedabad,2.0,2.00,0.20,0.00,23.068325,71.981420
3,4,Dhandhuka,Ahmedabad,23.0,1.00,0.01,0.00,22.381636,71.979141
4,5,Dholka,Ahmedabad,42.0,2.20,0.20,0.00,22.726607,72.441186
5,6,Viramgam,Ahmedabad,42.0,5.00,1.24,0.00,23.122452,72.048395
6,7,Kerala,Ahmedabad,97.3,9.50,1.13,0.00,22.779521,72.328501


In [103]:
# multiple markers using dataframe
# there is an example below using apply function instead of loop
fig=Figure(width=1000,height=800)
# create map
map_industry = folium.Map(location=[22.3, 71.5724], zoom_start=7.5, tiles='Stamen Terrain')

# Industry areas locations
for (index, row) in df_ind_area_nn.iterrows():
    folium.Marker(location=[row.loc['Latitude'], row.loc['Longitude']], 
                  popup=row.loc['Estate'] + ' ' + row.loc['District'], 
                  tooltip='click',
                  icon=DivIcon(
                                icon_size=(150,36),
                                icon_anchor=(7,20),
                                html=f'<div style="font-size: 13pt; color : red">{index}</div>',
                                )).add_to(map_industry)
    # map_industry.add_child(folium.Marker([row.loc['Latitude'], row.loc['Longitude']]))
    
# display map    
fig.add_child(map_industry)

# Match both coordinates

In [90]:
import numpy as np
arr_lat = df_ind_area_nn['Latitude'].values
arr_lon = df_ind_area_nn['Longitude'].values

arr_lat = arr_lat.reshape(-1, 1)
arr_lon = arr_lon.reshape(-1, 1)

df_fc_latlon = df_fc[['Latitude', 'Longitude']]
lat_mat = arr_lat - df_fc_latlon.values[:, 0].reshape(-1,1).T
lon_mat = arr_lon - df_fc_latlon.values[:, 1].reshape(-1,1).T

lat_mat = lat_mat ** 2
lon_mat = lon_mat ** 2


d_matrix_ind_loc = np.sqrt(lat_mat + lon_mat).argmin(axis=1)
d_matrix_ind_loc

array([ 809, 1278, 1060,  744, 1004,  818,  818,  882, 1490, 1488, 1292,
       1546, 1292, 1425, 1179, 1015,  823, 1063, 1119, 1119,  891, 1172,
       1068, 1229, 1290,  751,  683,  683,  754,  497,  495, 2373, 2367,
       2392, 2375, 2412, 2390, 2383, 2297, 2319, 2261, 1144, 1381, 1195,
       1254, 1581, 1261, 1585, 1395, 1746,  985, 1744, 1639, 1639, 1396,
       1456, 2228, 2187, 2009, 2045, 2142, 2184, 2223, 2287, 1920, 2214,
       1811, 1861, 1701, 1653, 1925, 2175, 2042,  111,  132,  103,  977,
        721,  722,  648,  972, 1716, 1542, 1878, 1937, 1718, 1773, 1824,
       1664, 1882, 1937, 1880, 1823, 1611, 1947, 1887, 1724, 1724, 2024,
       1965, 1965, 1840, 1677, 1157, 1268,  935, 1153,  868, 1161, 1050,
        388,  438,  388,  613,  388,  259,  341,  221, 1020, 1028, 1028,
        505,  813], dtype=int64)

In [92]:
df_fc.iloc[d_matrix_ind_loc, :].head()

,Index,Latitude,Longitude,2010,2011,2012,2013,2014,2015,2016,2017
809,809,23.05617,71.96844,205.800293,209.120041,123.753914,188.607239,193.270874,219.268951,170.129959,201.188522
1278,1278,22.41137,71.96844,55.974243,43.454185,35.976013,43.097969,39.536823,49.120884,42.676819,44.895374
1060,1060,22.73377,72.44619,239.723084,220.505890,187.552765,211.039352,178.114487,246.398056,196.065643,197.363495
744,744,23.13677,72.04807,290.464630,236.578354,213.973709,244.213089,209.262894,265.303253,221.294220,229.672760
1004,1004,22.81437,72.36657,285.816650,214.206207,188.136673,223.894516,184.556412,234.186188,184.094040,177.884476


In [104]:
df_ind_area_nn_mapped = df_ind_area_nn.copy()
df_ind_area_nn_mapped['idx_map'] = d_matrix_ind_loc
df_ind_area_nn_mapped.head()

,Sr.No.,Estate,District,Size in Ha.,Power Supply (MVA),Water Supply (MLD),Drainage (MLD),Latitude,Longitude,idx_map
1,2,Vani,Ahmedabad,2.0,2.00,0.20,0.00,23.068325,71.981420,809
3,4,Dhandhuka,Ahmedabad,23.0,1.00,0.01,0.00,22.381636,71.979141,1278
4,5,Dholka,Ahmedabad,42.0,2.20,0.20,0.00,22.726607,72.441186,1060
5,6,Viramgam,Ahmedabad,42.0,5.00,1.24,0.00,23.122452,72.048395,744
6,7,Kerala,Ahmedabad,97.3,9.50,1.13,0.00,22.779521,72.328501,1004


In [116]:
df_coords = df_fc.iloc[:, :3].copy()
df_coords = df_coords.merge(df_ind_area_nn_mapped[['idx_map', 'Latitude', 'Longitude', 'Estate', 'District']], 
                            left_on='Index', right_on='idx_map', how='left', suffixes=('', '_ind'))
df_coords = df_coords[~df_coords['Latitude_ind'].isna()]

In [117]:
# multiple markers using dataframe
# there is an example below using apply function instead of loop
fig=Figure(width=1000,height=800)
# create map
map_industry = folium.Map(location=[22.3, 71.5724], zoom_start=7.5, tiles='Stamen Terrain')

# Industry areas locations
for (index, row) in df_coords.iterrows():
    folium.Marker(location=[row.loc['Latitude'], row.loc['Longitude']], 
                  popup=str(row.loc['Estate']) + ' ' + str(row.loc['District']), 
                  tooltip='click',
                  icon=folium.Icon(color='blue')).add_to(map_industry)
    
    folium.Marker(location=[row.loc['Latitude_ind'], row.loc['Longitude_ind']], 
                  popup=str(row.loc['Estate']) + ' ' + str(row.loc['District']), 
                  tooltip='click',
                  icon=folium.Icon(color='red')).add_to(map_industry)
    # map_industry.add_child(folium.Marker([row.loc['Latitude'], row.loc['Longitude']]))
    
# display map    
fig.add_child(map_industry)